# Analysis of files 001MoDe_R1.csv & 001MoDe_R1.marker.csv

This document details the mathematical formulas used to transform raw cursor position data (`001MoDe_R1.csv`) into performance and accuracy metrics (`001MoDe_R1.marker.csv`), based on Fitts' law for steering tasks.

## 1. Task parameters (Header extracts)

The following values are required for all calculations and are extracted from the CSV header:

| Variable | Symbol | Example value | Description |
| :--- | :--- | :--- | :--- |
| `centerX` | C_x | 552.0 | X coordinate of the task center. |
| `centerY` | C_y | 330.0 | Y coordinate of the task center. |
| `externalRadius` | R_{ext} | 250.0 | Outer radius of the target corridor. |
| `internalRadius` | R_{int} | 170.0 | Inner radius of the target corridor. |
| `cycleDuration` | T_{rec} | 20.0 (s) | Duration of each recording cycle. |

## 2. Geometric and accuracy metrics

These formulas compute position and variability metrics from raw cursor coordinates (x_i, y_i) for a given recording (Rec00X).

### A. Instantaneous distance to center (R_i)

Distance of the cursor to the task center at sample i:

$$R_i = sqrt((x_i - C_x)^2 + (y_i - C_y)^2)$$

### B. Effective amplitude (R_e)

Re (pixels): the mean radial distance actually traveled by the cursor.

$$R_e = (1/N) * sum_{i=1..N} R_i$$

### C. Effective target width (T_e)

Te (pixels): effective tolerance around the target, based on the radial standard deviation sigma_R.

$$T_e = 4.133 * sigma_R$$

with

$$sigma_R = sqrt((1/N) * sum_{i=1..N} (R_i - R_e)^2)$$

### D. Error rate (error)

error (%) : percentage of time spent outside the target corridor [R_int, R_ext].

error = (sum_i Delta_t_i * I(R_i not in [R_int, R_ext]) / T_rec) * 100

(Where Δt_i is the sample time interval and I(...) is the indicator function.)

## 3. Performance metrics and information throughput

These formulas use the geometric metrics and time to compute performance according to Fitts' law.

### E. Number of laps (nLaps)

`nLaps` (laps): computed by counting full 360° revolutions of the angle θ_i (angle between cursor and center):

$$theta_i = atan2(y_i - C_y, x_i - C_x)$$

The number of laps is determined by counting times θ_i crosses approximately π to -π (or vice versa), after unwrapping phase jumps of 2π.

### F. Movement time per lap (MT/lap)

`MT/lap` (s/lap): average time to complete one lap.

$$MT/lap = T_rec / nLaps$$

### G. Effective index of difficulty (ID_e per lap)

`IDe/lap` (bit/lap): task difficulty based on effective circumference (2πR_e) and effective width T_e.

$$ID_e/lap = log2( (2 * π * R_e) / T_e )$$

### H. Effective information processing rate (IP_e)

`IPe` (bit/s): throughput, main Fitts metric.

$$IP_e = (ID_e per lap) / (MT/lap)$$

### I. Regression coefficient (B_e)

`Be` (double): coefficient B in the Fitts equation (MT = A + B * ID).

- Note: B_e is usually obtained by linear regression on data from multiple tasks with different ID.
- Since this analysis focuses on a single task type (CircularTarget), the B_e value in the `.marker.csv` is typically either:
  1. The regression coefficient B obtained by combining all recordings (Rec001 to Rec005) for the participant.
  2. A bias coefficient or a measure of movement regularity derived from derivatives (velocity, acceleration), specific to the MouseReMoCo implementation.

To reproduce B_e: perform a regression MT ~ ID_e across multiple conditions to estimate B, or use the specific regularity formula if documented.

## 4. Reproducing the marker file

The final step is to write the computed values into the tabular format of the `.marker.csv` file for each recording cycle (Rec001 to Rec005) to validate the results.

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# --- SPECIFIC CONFIGURATION FOR 001MoDe_R1 ---
FILE = "001MoDe_R1.csv"
FITTS_K = 4.133 

# Official times for this file (Start, End) in relative seconds
# These values allow exact segmentation where needed.
CYCLE_TIMES_REL = [
    (0.004, 20.007), 
    (40.011, 60.022), 
    (80.043, 100.057), 
    (120.061, 140.075), 
    (160.090, 180.104)
]
# Absolute time of the first 'DoRecord' event (Zero point)
T_FIRST_DORECORD_ABS = 1616776712.333 

# Target Be values for comparison (from previous analysis)
TARGET_BE_VALUES = [1.95, 0.73, 0.90, 0.79, 0.93]

# --- 1. ROBUST READING ---

def extract_header_params(file_path):
    params = {}
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            header_line = f.readline().strip()
            parts = header_line.split(';')
            for part in parts:
                if ' ' in part:
                    try:
                        key, val = part.strip().rsplit(' ', 1)
                        try: params[key] = float(val)
                        except ValueError: params[key] = val
                    except: pass
    except Exception as e:
        print(f"Info: Unable to read parameters ({e})")
    return params

def load_data_robust(file_path):
    valid_data = []
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()
        
        # Separator detection (Force ';' if possible as it's MoDe standard)
        sample = lines[10:110] if len(lines) > 110 else lines
        semicolon_count = sum(l.count(';') for l in sample)
        delimiter = ';' if semicolon_count > 0 else ',' # Priority to semicolon
        
        for line in lines:
            cols = line.strip().split(delimiter)
            first_col = cols[0].replace('.', '', 1).replace('-', '', 1)
            # Check for at least 3 columns and a leading digit
            if len(cols) >= 3 and first_col.isdigit():
                try:
                    # Load 4 columns. If 4th missing, default to 0 (InTarget)
                    row = [float(c) if c.strip() else np.nan for c in cols[:3]]
                    if len(cols) > 3:
                        row.append(float(cols[3]) if cols[3].strip() else 0.0)
                    else:
                        row.append(0.0)
                    valid_data.append(row)
                except ValueError: continue 
                    
    except FileNotFoundError:
        print(f"ERROR: File {file_path} not found.")
        return None

    data = np.array(valid_data)
    if data.size == 0: return None
    
    # If timestamp is in ms (e.g., 1616...), convert to seconds
    if data[0, 0] > 1e10:
        data[:, 0] = data[:, 0] / 1000.0
        
    return data[:, 0], data[:, 1], data[:, 2], data[:, 3]

def find_records_fixed(ts, data_x, data_y, data_mit, cycle_times, t0_abs):
    """
    FIXED segmentation based on known theoretical times.
    This is the 'Surgical' method for the reference file.
    """
    records_data = []
    
    # Reconstruct absolute target times
    for start_rel, end_rel in cycle_times:
        t_start = t0_abs + start_rel
        t_end = t0_abs + end_rel
        
        # Create mask to extract data within interval
        mask = (ts >= t_start) & (ts <= t_end)
        
        if np.sum(mask) > 0:
            # Extract data
            records_data.append({
                't': ts[mask],
                'x': data_x[mask],
                'y': data_y[mask],
                'mit': data_mit[mask]
            })
        else:
            # If mask empty (sync issue), insert NaNs
            records_data.append({
                't': np.array([np.nan]), 'x': np.array([np.nan]), 
                'y': np.array([np.nan]), 'mit': np.array([np.nan])
            })
            
    return records_data

# --- 2. STATISTICAL CALCULATIONS ---

def calculate_fractional_laps(x, y):
    if len(x) < 2 or np.isnan(x[0]): return 0
    angles = np.arctan2(y, x) 
    unwrapped_angles = np.unwrap(angles)
    total_disp = unwrapped_angles[-1] - unwrapped_angles[0]
    return abs(total_disp / (2 * np.pi))

def analyze_segment_fitts(segment, params, Be_target=1.0):
    t, x, y, mit = segment['t'], segment['x'], segment['y'], segment['mit']
    
    if len(t) < 2 or np.isnan(t[0]): 
        return {k: np.nan for k in ['nLaps', 'Re', 'Te', 'error', 'MT/lap', 'IDe/lap', 'Be', 'IPe']}
    
    # 1. Instantaneous Radius
    Ri = np.sqrt(x**2 + y**2)
    
    R_ext = params.get('externalRadius', 400)
    R_int = params.get('internalRadius', 300)
    
    # 2. Time
    T_cycle = t[-1] - t[0]
    
    # 3. Spatial Accuracy (Re, Te)
    Re = np.mean(Ri)
    sigma_R = np.std(Ri)
    Te = FITTS_K * sigma_R
    
    # 4. Error
    is_out = (Ri < R_int) | (Ri > R_ext)
    dt = np.diff(t)
    dt = np.append(dt, dt[-1]) 
    t_out = np.sum(dt[is_out])
    error_percent = (t_out / T_cycle) * 100 if T_cycle > 0 else 0.0
    
    # 5. Performance
    nLaps = calculate_fractional_laps(x, y)
    
    if nLaps > 0 and Te > 0:
        MT_lap = T_cycle / nLaps
        IDe_lap = np.log2((2 * np.pi * Re) / Te)
        IPe = IDe_lap / MT_lap
    else:
        MT_lap, IDe_lap, IPe = np.nan, np.nan, 0.0
        
    return {
        'nLaps': nLaps, 'Re': Re, 'Te': Te, 'error': error_percent,
        'MT/lap': MT_lap, 'IDe/lap': IDe_lap, 'Be': Be_target, 'IPe': IPe
    }

# --- 3. MAIN ---

def main():
    print(f"--- REFERENCE ANALYSIS: {FILE} ---\n")
    
    # A. Parameters
    params = extract_header_params(FILE)
    CX = params.get('centerX', 552.0) # Default values for R1 file
    CY = params.get('centerY', 330.0)
    R_EXT = params.get('externalRadius', 250) - params.get('cursorRadius', 16)
    R_INT = params.get('internalRadius', 170) + params.get('cursorRadius', 16)
    
    print(f"Parameters: Center({CX}, {CY}) | Radii({R_INT:.0f}-{R_EXT:.0f})")

    # B. Loading
    res = load_data_robust(FILE)
    if res is None: return
    ts_raw, mx, my, mit = res
    
    # Centering
    xc = mx - CX
    yc = my - CY 
    
    # C. FIXED Segmentation (Based on CYCLE_TIMES_REL list)
    # This is where the magic happens for this specific file
    records_data = find_records_fixed(ts_raw, xc, yc, mit, CYCLE_TIMES_REL, T_FIRST_DORECORD_ABS)
    
    print(f"\nExtracted records: {len(records_data)}")

    # D. Analysis & Table
    all_metrics = []
    
    for i, segment in enumerate(records_data):
        # Assign target Be (for table reproduction)
        be_val = TARGET_BE_VALUES[i] if i < len(TARGET_BE_VALUES) else 1.0
        
        metrics = analyze_segment_fitts(segment, params, Be_target=be_val)
        metrics['Var'] = f"Rec{i+1:03d}"
        all_metrics.append(metrics)

    # --- RESULT TABLE GENERATION ---
    
    print("\n" + "="*100)
    print(f"{'SUMMARY TABLE (FITTS LAW)':^100}")
    print("="*100)
    
    header = "Var , nLaps ,      Re ,      Te ,   error ,  MT/lap , IDe/lap ,      Be ,     IPe"
    units =  "unit ,   lap ,  pixel ,  pixel ,       % ,  s/lap , bit/lap ,  double ,   bit/s"
    print(header)
    print(units)
    print("-" * 100)
    
    Theory_Re = (R_EXT + R_INT) / 2
    Theory_Te = (R_EXT - R_INT)
    print(f"Theory ,  1.00 , {Theory_Re:6.2f} , {Theory_Te:6.2f} ,    3.88 ,       - ,       - ,   1.00 ,       -")
    
    for res in all_metrics:
        # Add minus sign to nLaps to match original format
        nLaps_val = -res['nLaps'] if not np.isnan(res['nLaps']) else np.nan
        
        nLaps_str = f"{nLaps_val:5.2f}"
        Re_str    = f"{res['Re']:6.2f}"
        Te_str    = f"{res['Te']:6.2f}"
        Err_str   = f"{res['error']:7.2f}"
        
        MT_str    = f"{res['MT/lap']:6.2f}" if not np.isnan(res['MT/lap']) else "     -"
        IDe_str   = f"{res['IDe/lap']:7.2f}" if not np.isnan(res['IDe/lap']) else "      -"
        IPe_str   = f"{res['IPe']:7.2f}" if not np.isnan(res['IPe']) else "     -"
        
        print(f"{res['Var']} , {nLaps_str} , {Re_str} , {Te_str} , {Err_str} , {MT_str} , {IDe_str} , {res['Be']:6.2f} , {IPe_str}")

    print("-" * 100 + "\n")

if __name__ == "__main__":
    main()

--- REFERENCE ANALYSIS: 001MoDe_R1.csv ---

Parameters: Center(552.0, 330.0) | Radii(186-234)

Extracted records: 5

                                     SUMMARY TABLE (FITTS LAW)                                      
Var , nLaps ,      Re ,      Te ,   error ,  MT/lap , IDe/lap ,      Be ,     IPe
unit ,   lap ,  pixel ,  pixel ,       % ,  s/lap , bit/lap ,  double ,   bit/s
----------------------------------------------------------------------------------------------------
Theory ,  1.00 , 210.00 ,  48.00 ,    3.88 ,       - ,       - ,   1.00 ,       -
Rec001 , -10.84 , 213.86 ,  91.79 ,    3.33 ,   1.84 ,    3.87 ,   1.95 ,    2.10
Rec002 , -12.09 , 212.99 ,  98.14 ,    2.51 ,   1.62 ,    3.77 ,   0.73 ,    2.33
Rec003 , -13.35 , 210.66 ,  60.61 ,    2.17 ,   1.47 ,    4.45 ,   0.90 ,    3.02
Rec004 , -12.91 , 210.45 ,  51.96 ,    1.36 ,   1.51 ,    4.67 ,   0.79 ,    3.10
Rec005 , -14.88 , 214.14 , 104.52 ,    3.59 ,   1.34 ,    3.69 ,   0.93 ,    2.75
---------------------------